In [2]:
!pip install geopy

In [1]:
import numpy as np
import pandas as pd 
import math as m 
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from geopy.distance import geodesic

# Bases de données des accidents 

## Base de données accidents 

In [86]:
#df_accidents = pd.read_csv("BaseDeDonnées/Accidents_france/caracteristics.csv",encoding='latin-1') #2005-2016
df_accidents_2023 = pd.read_csv("BaseDeDonnées/Accidents_france/caract-2023.csv",sep=';') #2023
df_accidents_2022 = pd.read_csv("BaseDeDonnées/Accidents_france/carcteristiques-2022.csv",sep=';')#2022 

In [87]:
df_accidents_2022 = df_accidents_2022.rename(columns={'Accident_Id':'Num_Acc'})

In [88]:
#df_vehicules = pd.read_csv('BaseDeDonnées/Accidents_france/vehicles.csv')
df_vehicules_2023 = pd.read_csv('BaseDeDonnées/Accidents_france/vehicules-2023.csv',sep=';')
df_vehicules_2022 = pd.read_csv('BaseDeDonnées/Accidents_france/vehicules-2022.csv',sep=';')

## Filtre par département et zone urbaine 

In [89]:
df_accidents_grenoble_2023 = df_accidents_2023[df_accidents_2023['dep']=='38'] # 2023 
df_accidents_grenoble_2022 = df_accidents_2022[(df_accidents_2022['dep']== '38')] #2022

## Filtre sur les véhicules qui sont des vélos

In [90]:
df_vehicules_2023 = df_vehicules_2023[df_vehicules_2023['catv']==1]
df_vehicules_2022 = df_vehicules_2022[df_vehicules_2022['catv']==1]

## Merge avec Vehicules

In [92]:
df_accidents_grenoble_2023 = pd.merge(df_accidents_grenoble_2023,df_vehicules_2023, how='inner',on='Num_Acc')
df_accidents_grenoble_2022 = pd.merge(df_accidents_grenoble_2022,df_vehicules_2022, how='inner',on='Num_Acc')


In [75]:
df_accidents_grenoble_2023 = df_accidents_grenoble_2023[df_accidents_grenoble_2023['catv']==1]
df_accidents_grenoble_2022 = df_accidents_grenoble_2022[df_accidents_grenoble_2022['catv']==1]


In [95]:
print('Le nombre d\'accidents en 2023 était de : {} et de {} en 2022'.format(len(df_accidents_grenoble_2023),len(df_accidents_grenoble_2022)))

Le nombre d'accidents en 2023 était de : 68 et de 68 en 2022


## Indicateur : Nombre d'accidents impliquant les vélos 

L'étude de ces fichiers n'est pas terminée mais déjà on peut avoir une idée du nombre d'accidents impliquant des vélos et voir si l'on peut en tirer quelque chose. 
Entre 2005 - 2016  , il y a eu 448 accidents de vélo recensés par la police à Grenoble. 

### Ajout du fichier 'Place' 

In [10]:
df_places = pd.read_csv('BaseDeDonnées/Accidents_france_2005_2016/places.csv')
df_places

<ipython-input-10-d5dd9678af07>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_places = pd.read_csv('BaseDeDonnées/Accidents_france_2005_2016/places.csv')


,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1
0,201600000001,3.0,39,NaN,NaN,2.0,0.0,NaN,NaN,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0
1,201600000002,3.0,39,NaN,NaN,1.0,0.0,NaN,NaN,0.0,1.0,2.0,0.0,58.0,1.0,0.0,1.0,0.0
2,201600000003,3.0,1,NaN,NaN,2.0,2.0,NaN,NaN,0.0,1.0,3.0,0.0,68.0,2.0,0.0,3.0,99.0
3,201600000004,4.0,0,NaN,NaN,2.0,0.0,NaN,NaN,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,99.0
4,201600000005,4.0,0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839980,200500087950,4.0,0.0,0.0,NaN,2.0,3.0,NaN,NaN,0.0,1.0,1.0,0.0,80.0,1.0,0.0,1.0,3.0
839981,200500087951,4.0,0.0,0.0,NaN,2.0,2.0,NaN,NaN,0.0,1.0,2.0,0.0,70.0,1.0,0.0,1.0,0.0
839982,200500087952,4.0,0.0,0.0,NaN,1.0,2.0,NaN,NaN,0.0,1.0,1.0,0.0,75.0,1.0,0.0,1.0,3.0
839983,200500087953,4.0,0.0,0.0,NaN,2.0,0.0,NaN,NaN,0.0,1.0,1.0,0.0,150.0,1.0,0.0,1.0,0.0


#### Merge avec Places

In [11]:
df_accidents_grenoble = pd.merge(df_accidents_grenoble,df_places,how='left',on='Num_Acc')

In [12]:
df_accidents_grenoble 

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1
0,201600004855,16,7,20,1100,1,2,1,1.0,1.0,...,550.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,99.0
1,201600004856,16,9,30,830,1,1,1,1.0,5.0,...,300.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,99.0
2,201600004891,16,9,25,1545,1,2,1,1.0,1.0,...,NaN,0.0,1.0,4.0,54.0,0.0,1.0,0.0,1.0,99.0
3,201600004893,16,11,23,1400,1,1,1,1.0,1.0,...,NaN,2.0,2.0,1.0,0.0,75.0,1.0,0.0,1.0,99.0
4,201600004901,16,3,30,700,2,2,1,1.0,1.0,...,NaN,0.0,1.0,1.0,0.0,60.0,1.0,0.0,1.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,200500056082,5,9,14,1900,1,2,6,1.0,6.0,...,0.0,0.0,2.0,2.0,0.0,0.0,1.0,5.0,1.0,3.0
444,200500056084,5,9,24,2230,5,2,1,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0
445,200500063470,5,10,27,1315,1,1,1,6.0,7.0,...,0.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0
446,200500071206,5,11,16,1530,1,2,1,8.0,3.0,...,0.0,1.0,1.0,1.0,0.0,109.0,1.0,0.0,1.0,0.0


#### Merge avec Users 

In [13]:
df_accidents_grenoble = pd.merge(df_accidents_grenoble,df_users, how='left',on='Num_Acc')


In [14]:
df_accidents_grenoble = df_accidents_grenoble[df_accidents_grenoble['catu']==1]

In [15]:
df_accidents_grenoble = df_accidents_grenoble.drop(columns=['catu'])

In [16]:
df_accidents_grenoble

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,place,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh
0,201600004855,16,7,20,1100,1,2,1,1.0,1.0,...,1.0,1,2,5.0,11.0,0.0,0.0,0.0,1959.0,A01
1,201600004855,16,7,20,1100,1,2,1,1.0,1.0,...,1.0,3,1,5.0,22.0,0.0,0.0,0.0,1946.0,B02
2,201600004856,16,9,30,830,1,1,1,1.0,5.0,...,1.0,2,2,1.0,21.0,0.0,0.0,0.0,1970.0,A01
3,201600004856,16,9,30,830,1,1,1,1.0,5.0,...,1.0,1,2,1.0,11.0,0.0,0.0,0.0,1957.0,B02
4,201600004856,16,9,30,830,1,1,1,1.0,5.0,...,1.0,1,1,5.0,11.0,0.0,0.0,0.0,1962.0,C03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,200500063470,5,10,27,1315,1,1,1,6.0,7.0,...,1.0,3,1,5.0,0.0,0.0,0.0,0.0,1991.0,A01
902,200500071206,5,11,16,1530,1,2,1,8.0,3.0,...,1.0,1,2,5.0,11.0,0.0,0.0,0.0,1982.0,A01
903,200500071206,5,11,16,1530,1,2,1,8.0,3.0,...,1.0,3,2,5.0,0.0,0.0,0.0,0.0,1994.0,B02
905,200500071245,5,11,16,1600,1,2,1,1.0,1.0,...,1.0,1,2,1.0,11.0,0.0,0.0,0.0,1941.0,A01


In [17]:
df_accidents_grenoble.dtypes

Num_Acc      int64
an           int64
mois         int64
jour         int64
hrmn         int64
lum          int64
agg          int64
int          int64
atm        float64
col        float64
com        float64
adr         object
gps         object
lat        float64
long        object
dep          int64
senc       float64
catv         int64
occutc       int64
obs        float64
obsm       float64
choc       float64
manv       float64
catr       float64
voie        object
v1         float64
v2          object
circ       float64
nbv        float64
pr         float64
pr1        float64
vosp       float64
prof       float64
plan       float64
lartpc     float64
larrout    float64
surf       float64
infra      float64
situ       float64
env1       float64
place      float64
grav         int64
sexe         int64
trajet     float64
secu       float64
locp       float64
actp       float64
etatp      float64
an_nais    float64
num_veh     object
dtype: object

In [18]:
df_accidents_grenoble = df_accidents_grenoble.drop(columns=['jour',
                                                            'mois',
                                                            'hrmn',
                                                            'dep',
                                                            'com',
                                                            'adr',
                                                            'gps',
                                                            'voie',
                                                            'v1',
                                                            'v2',
                                                           'pr',
                                                           'pr1',
                                                           'lartpc',
                                                           'larrout',
                                                           'catv',
                                                           'sexe',
                                                           'etatp',
                                                            'num_veh',
                                                            'an_nais',
                                                            'lat',
                                                            'long'
                                                           ])

In [19]:
df_accidents_grenoble

,Num_Acc,an,lum,agg,int,atm,col,senc,occutc,obs,...,surf,infra,situ,env1,place,grav,trajet,secu,locp,actp
0,201600004855,16,1,2,1,1.0,1.0,2.0,0,0.0,...,1.0,0.0,1.0,99.0,1.0,1,5.0,11.0,0.0,0.0
1,201600004855,16,1,2,1,1.0,1.0,2.0,0,0.0,...,1.0,0.0,1.0,99.0,1.0,3,5.0,22.0,0.0,0.0
2,201600004856,16,1,1,1,1.0,5.0,2.0,0,0.0,...,1.0,0.0,1.0,99.0,1.0,2,1.0,21.0,0.0,0.0
3,201600004856,16,1,1,1,1.0,5.0,2.0,0,0.0,...,1.0,0.0,1.0,99.0,1.0,1,1.0,11.0,0.0,0.0
4,201600004856,16,1,1,1,1.0,5.0,2.0,0,0.0,...,1.0,0.0,1.0,99.0,1.0,1,5.0,11.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,200500063470,5,1,1,1,6.0,7.0,0.0,0,8.0,...,1.0,0.0,3.0,0.0,1.0,3,5.0,0.0,0.0,0.0
902,200500071206,5,1,2,1,8.0,3.0,0.0,0,0.0,...,1.0,0.0,1.0,0.0,1.0,1,5.0,11.0,0.0,0.0
903,200500071206,5,1,2,1,8.0,3.0,0.0,0,0.0,...,1.0,0.0,1.0,0.0,1.0,3,5.0,0.0,0.0,0.0
905,200500071245,5,1,2,1,1.0,1.0,0.0,0,0.0,...,1.0,5.0,1.0,0.0,1.0,1,1.0,11.0,0.0,0.0


In [20]:
df_accidents_grenoble.isna().sum()

Num_Acc    0
an         0
lum        0
agg        0
int        0
atm        0
col        0
senc       0
occutc     0
obs        2
obsm       2
choc       2
manv       2
catr       0
circ       1
nbv        3
vosp       5
prof       3
plan       3
surf       1
infra      5
situ       5
env1       5
place      0
grav       0
trajet     0
secu       3
locp       0
actp       0
dtype: int64

## Deuxième indicateur : comptage permanent vélo 

In [36]:
import pandas as pd

df=pd.read_csv("BaseDeDonnées/comptages_velos_permanents.csv")
df_vf=df[df['territoire'] == 'Grenoble Alpes Metropole']

#print(df.info())
#print(df.head(3))
df_vf.head()

,id,nom_post,nom_comm,code_insee,modes,date_instal,localisation,type_compteur,type_axe,tmj_2019,...,jour_de_pointe_2021,jour_de_pointe_2020,jour_de_pointe_2019,date_pointe_2022,date_pointe_2021,date_pointe_2020,territoire,date_pointe_2019,fid,geo_point_2d
2,28,(Echirolles) rue Victor Hugo,Échirolles,38151,Velos,20201112,Rue Victor Hugo,Boucle_ZELT,chronovélo,NaN,...,1135.0,528.0,NaN,20221018,20210907.0,20201118.0,Grenoble Alpes Metropole,NaN,3,"45.15498351431255, 5.71241623986171"
5,31,(SMLV) Pont d'Oxford - Afficheur,Saint-Martin-le-Vinoux,38423,Velos,20220210,Pont d'Oxford,ZELT+Afficheur,voie verte,NaN,...,NaN,NaN,NaN,20221018,NaN,NaN,Grenoble Alpes Metropole,NaN,6,"45.207988000018965, 5.702445999999999"
6,30,(Saint Martin d'Hères) Campus universitaire,Saint-Martin-d'Hères,38421,Velos,20220406,Campus Universitaire,Boucle_ZELT,chronovélo,NaN,...,NaN,NaN,NaN,20221018,NaN,NaN,Grenoble Alpes Metropole,NaN,7,"45.186215000019004, 5.769533999999999"
7,32,(Pont de Claix) PEM l'Etoile - Afficheur,Le Pont-de-Claix,38317,Velos,20210927,L'Etoile,ZELT+Afficheur,chronovélo,NaN,...,521.0,NaN,NaN,20221011,20211014.0,NaN,Grenoble Alpes Metropole,NaN,8,"45.13781800001914, 5.702301000000001"
8,26,(Meylan) Mairie / Afficheur,Meylan,38229,Velos,20200918,Mairie de Meylan,ZELT+Afficheur,chronovélo,NaN,...,990.0,964.0,NaN,20221018,20210908.0,20200923.0,Grenoble Alpes Metropole,NaN,9,"45.20945100001895, 5.779065999999999"


In [ ]:
# Afficher ou vérifier les premières lignes du nouveau DataFrame
colonnes=['nom_post', 'modes','localisation','type_compteur','type_axe','tmj_2019','tmj_2020','tmj_2021','tmj_2022']
colonnes_bis=['nom_post','localisation','type_axe','tmj_2019','tmj_2020','tmj_2021','tmj_2022']

print(df_vf[colonnes_bis].head(10))

#### Comme tous les compteurs n'ont pas été installés au même moment, il existe beaucoup de valeurs manquantes (NaN), il sera alors nécessaire de les ignorer au moment des analyses car il ne conviendrait pas de les remplacer par des zéros (fausseraient les résultats) de supprimer les lignes associées (trop nombreuses)
#### Ce qu'il faudrait c'est donc au moment de faire les analyses, passer outre ces valeurs vides (si l'on cherche a avoir des moyennes par années ou par ville par exemple)

## Troisième indicateur : Kilomètre piste cyclable

In [38]:
df = pd.read_csv('BaseDeDonnées/pistes_cyclables.xls')

In [39]:
df.head()

,ogc_fid,geo_point_2d,anne_maj,lametro,type,name,geo_shape
0,1489,"45.17402,5.74223",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.74223,4..."
1,1490,"45.17272,5.73701",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73701,4..."
2,1491,"45.17273,5.73488",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73488,4..."
3,1492,"45.17315,5.73476",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73476,4..."
4,1625,"45.20183,5.75957",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.75957,4..."
...,...,...,...,...,...,...,...
1197,2813,"45.15938,5.72401",2022.0,1,velodifficile,NaN,"{""type"":""LineString"",""coordinates"":[[5.72401,4..."
1198,2810,"45.20988,5.83977",2021.0,1,velodifficile,NaN,"{""type"":""LineString"",""coordinates"":[[5.83977,4..."
1199,1637,"45.19119,5.72127",2022.0,1,velodifficile,NaN,"{""type"":""LineString"",""coordinates"":[[5.72127,4..."
1200,1639,"45.18615,5.72467",2022.0,1,velodifficile,NaN,"{""type"":""LineString"",""coordinates"":[[5.72467,4..."


### Vérification des type LineString

In [ ]:
# Fonction pour vérifier le type de géométrie
def is_linestring(geo_shape):
    try:
        shape = json.loads(geo_shape)
        return shape['type'] == 'LineString'
    except:
        return False

# Vérifier si toutes les géométries sont des LineString
df['is_linestring'] = df['geo_shape'].apply(is_linestring)

# Résumé des résultats
print("Résumé des types de géométries :")
print(df['is_linestring'].value_counts())

# Identifier les géométries non conformes, s'il y en a
non_linestring_rows = df[~df['is_linestring']]
if not non_linestring_rows.empty:
    print("Lignes avec des géométries non conformes :")
    print(non_linestring_rows)
else:
    print("Toutes les géométries sont des LineString.")

### Calcul des distances de la piste cyclable

In [ ]:
# Fonction pour calculer la distance d'une piste cyclable à partir de la colonne geo_shape
def calculate_distance(geo_shape):
    try:
        # Charger les coordonnées GeoJSON
        shape = json.loads(geo_shape)
        if shape['type'] == 'LineString':
            coordinates = shape['coordinates']
            # Calculer la distance totale entre les points consécutifs
            distances = [
                geodesic(coordinates[i], coordinates[i+1]).kilometers
                for i in range(len(coordinates) - 1)
            ]
            return sum(distances)
    except:
        return None  # Retourner None si un problème survient

# Ajouter une colonne pour les distances
df['distance_km'] = df['geo_shape'].apply(calculate_distance)

# Enregistrer le DataFrame avec la nouvelle colonne dans un nouveau fichier CSV
df.to_csv('fichier_avec_distances.csv', index=False)

# Afficher les premières lignes pour vérifier
print(df.head())

# Étude comparative étudiants UGA + INP 

In [19]:
df_uga = pd.read_csv('BaseDeDonnées/Etude_UGA/déplacements_UGA_output.csv',sep=';')

In [20]:
df_uga

,N°Obs,1. Dans quel établissement principal êtes-vous inscrit.e ?,2. En quel diplôme êtes-vous inscrit.e ?,4. Quel est votre site d’enseignement principal ?,"6. Aurez-vous une interruption de présence sur votre lieu d'enseignement cette année (ex : stage, césure...) ?","8. En moyenne, lorsque vous n'êtes pas en vacances, combien de jours par semaine vous rendez-vous sur votre lieu d’études ?",9. Quels modes de transport utilisez-vous pour vous rendre sur votre lieu d'études au cours de la journée-type la plus fréquent,37. Clé,38. Date de saisie,40. Date de dernier enregistrement,41. Temps de saisie,42. Langue,43. Progression,44. Dernière question saisie,45. Origine,46. Appareil utilisé pour la saisie
0,1,Grenoble INP - UGA,Non renseignée,Non renseignée,Non renseignée,-1,Non renseignée,L9SQ-APB5,15/03/2023 11:15:11,15/03/2023 11:15:21,"10,4383941",FR,Terminé,NaN,Direct,PC
1,2,UGA,Licence,Domaine Universitaire (Saint Martin d'Hères / ...,Oui,5,Marche;Tramway,K998-RC5G,15/03/2023 11:13:58,15/03/2023 11:15:25,"87,0446392",FR,Terminé,NaN,Direct,PC
2,3,ENSAG - UGA,Non renseignée,Non renseignée,Non renseignée,-1,Non renseignée,KWKN-UXBB,15/03/2023 11:15:31,15/03/2023 11:15:42,"11,2867948",FR,Terminé,NaN,Direct,PC
3,4,UGA,Licence,Domaine Universitaire (Saint Martin d'Hères / ...,Non,5,Vélo électrique;Voiture,T7YH-4DCZ,15/03/2023 11:14:26,15/03/2023 11:16:06,"99,9477878",FR,Terminé,NaN,Direct,Smartphone
4,5,Grenoble INP - UGA,Non renseignée,Non renseignée,Non renseignée,-1,Non renseignée,HT4V-XVQZ,15/03/2023 11:16:48,15/03/2023 11:17:07,"18,6326389",FR,Terminé,NaN,Direct,PC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,2140,UGA,Licence,Domaine Universitaire (Saint Martin d'Hères / ...,Non,5,Tramway,5LCZ-6WS2,09/06/2023 11:10:50,09/06/2023 11:14:34,"224,1824294",FR,Terminé,NaN,Direct,PC
2140,2141,UGA,Etudes de médecine ou de pharmacie (hors Licen...,Campus Santé (La Tronche),Oui,1,Marche;Vélo,M5K3-DMGW,28/06/2023 14:23:20,28/06/2023 14:36:31,"790,4706694",FR,Terminé,NaN,Direct,PC
2141,2142,UGA,Etudes de médecine ou de pharmacie (hors Licen...,Campus Santé (La Tronche),Oui,0,Train,7EKD-JXSN,03/07/2023 21:51:22,03/07/2023 21:52:55,"93,1100442",FR,Terminé,NaN,Direct,PC
2142,2143,Grenoble INP - UGA,Non renseignée,Non renseignée,Non renseignée,-1,Non renseignée,BY46-ZYUW,30/07/2023 12:01:00,30/07/2023 12:01:08,"8,5781704",FR,Terminé,NaN,Direct,PC
